# TUGAS MODUL 2: Transformasi ke Paradigma Fungsional
**Fatih - 132**

## Deskripsi Tugas
Mengubah seluruh fungsi dari program **Manajemen Proyek IoT** (demo.ipynb modul 1) agar memenuhi paradigma fungsional:
- **Pure Function**: Tidak ada side effect, tidak mengubah state global
- **Deklaratif**: Menggunakan ekspresi, comprehension, higher-order functions
- **Immutability**: Data tidak dimutasi, selalu mengembalikan struktur baru

## Strategi Transformasi
1. Semua fungsi menerima data sebagai parameter, mengembalikan hasil baru (tidak mutasi global)
2. Pisahkan logika bisnis dari I/O (input/output)
3. Gunakan list/dict comprehension, `map`, `filter`, `reduce`
4. State management melalui return values, bukan global mutation
5. Validasi dan transformasi data menggunakan pure functions

---

## 1. Transformasi: Fungsi Register

### BEFORE (Imperatif, Impure)
```python
def register():
    print("\n--- Halaman Register ---")
    while True:
        id_baru = input("Masukkan ID baru: ")
        if id_baru in data_akun:
            print("ID sudah terdaftar")
        elif not id_baru:
            print("ID tidak boleh kosong")
        else:
            break
    
    password_baru = input("Masukkan password: ")
    data_akun[id_baru] = password_baru
```

**Masalah:**
- Mengubah variabel global (`data_akun`, `data_profil`, `data_proyek_iot`)
- Sulit di-test (perlu mock input/output)
- Tidak bisa diprediksi hasilnya tanpa eksekusi
- Logika bisnis tercampur dengan I/O

### AFTER (Fungsional, Pure)
Pisahkan menjadi pure functions untuk validasi dan transformasi data:

In [ ]:
def validasi_id_unik(user_id: str, existing_accounts: dict) -> tuple[bool, str]:
    if not user_id:
        return (False, "ID tidak boleh kosong")
    if user_id in existing_accounts:
        return (False, "ID sudah terdaftar")
    return (True, "")

def buat_akun_baru(user_id: str, password: str, nama: str, jabatan: str,
        data_akun: dict, data_profil: dict, data_proyek: dict) -> tuple[dict, dict, dict]:
    
    new_akun = {**data_akun, user_id: password}
    new_profil = {**data_profil, user_id: {'Nama': nama, 'Jabatan': jabatan}}
    new_proyek = {**data_proyek, user_id: []}
    return (new_akun, new_profil, new_proyek)

def demo_register_functional():
    akun = {'admin': 'admin123'}
    profil = {'admin': {'Nama': 'Admin', 'Jabatan': 'PM'}}
    proyek = {'admin': []}
    
    user_input = {
        'id': 'user02',
        'password': 'pass02',
        'nama': 'Budi',
        'jabatan': 'Engineer'
    }
    
    is_valid, msg = validasi_id_unik(user_input['id'], akun)
    if not is_valid:
        print(f"Error: {msg}")
        return
    
    new_akun, new_profil, new_proyek = buat_akun_baru(
        user_input['id'], user_input['password'],
        user_input['nama'], user_input['jabatan'],
        akun, profil, proyek
    )
    
    print("Registrasi berhasil!")
    print("Akun baru:", new_akun)
    print("Profil baru:", new_profil)

demo_register_functional()

Registrasi berhasil!
Akun baru: {'admin': 'admin123', 'user02': 'pass02'}
Profil baru: {'admin': {'Nama': 'Admin', 'Jabatan': 'PM'}, 'user02': {'Nama': 'Budi', 'Jabatan': 'Engineer'}}


**Penjelasan Transformasi:**
1. **Pemisahan Concern**: Validasi (`validasi_id_unik`) terpisah dari transformasi data (`buat_akun_baru`)
2. **Immutability**: Menggunakan spread operator `{**dict}` untuk membuat dictionary baru, bukan mutasi
3. **Pure Function**: Semua fungsi deterministik, input sama → output sama
4. **Testability**: Mudah di-test tanpa mock karena tidak ada I/O internal
5. **No Side Effect**: Tidak mengubah variabel di luar scope fungsi

---

## 2. Transformasi: Fungsi Login

### BEFORE (Imperatif, Impure)
```python
def login():
    print("\n--- Halaman Login ---")
    percobaan = 0
    while percobaan < 3:
        id_login = input("ID: ")
        password = input("Password: ")
        if id_login in data_akun and data_akun[id_login] == password:
            print("Login berhasil!")
            return id_logi
        else:
            print("Salah")
            percobaan += 1
    print("Gagal 3x")
    return None
```

**Masalah:**
- I/O tercampur dengan logik autentikasi
- Menggunakan state lokal (`percobaan`) yang di-mutasi
- Tidak bisa di-test tanpa mock input

### AFTER (Fungsional, Pure)

In [ ]:
def autentikasi(user_id: str, password: str, data_akun: dict) -> bool:
    return user_id in data_akun and data_akun[user_id] == password

def ambil_profil(user_id: str, data_profil: dict) -> dict:
    return data_profil.get(user_id, {'Nama': 'Unknown', 'Jabatan': 'Unknown'})

def demo_login_functional():
    akun = {'admin': 'admin123', 'user01': 'pass01'}
    profil = {
        'admin': {'Nama': 'Admin Utama', 'Jabatan': 'PM'},
        'user01': {'Nama': 'Budi', 'Jabatan': 'Engineer'}
    }
    
    credentials = [
        ('admin', 'admin123'),
        ('admin', 'salah'),
        ('user01', 'pass01')
    ]
    
    for user_id, pwd in credentials:
        if autentikasi(user_id, pwd, akun):
            user_profil = ambil_profil(user_id, profil)
            print(f"Login berhasil: {user_profil['Nama']} ({user_profil['Jabatan']})")
        else:
            print(f"Login gagal untuk ID: {user_id}")

demo_login_functional();

✓ Login berhasil: Admin Utama (PM)
✗ Login gagal untuk ID: admin
✓ Login berhasil: Budi (Engineer)


**Penjelasan Transformasi:**
1. **Pure `autentikasi`**: Hanya memeriksa kredensial, return boolean (testable)
2. **Pure `ambil_profil`**: Menggunakan `.get()` dengan default (safe, tidak error)
3. **Separation of Concerns**: Logika autentikasi terpisah dari loop percobaan (loop ada di UI layer)
4. **Deterministic**: Input sama → output sama, tidak ada randomness atau state global

---

## 3. Transformasi: CRUD Perangkat IoT

### BEFORE (Imperatif, Impure)
```python
def kelola_proyek(id_login):
    while True:
        if pilihan == 1:
            id_p = input("ID: ")
            tipe_p = input("Tipe: ")
            data_proyek_iot[id_login].append({
                'id': id_p, 'tipe': tipe_p, 'status': 'Aktif'
            })
        elif pilihan == 3:
            idx = ...
            status_baru = input("Status: ")
            data_proyek_iot[id_login][idx]['status'] = status_baru
        elif pilihan == 4:
            idx = ...
            data_proyek_iot[id_login].pop(idx)
```

**Masalah:**
- Mutasi langsung list global (side effect)
- Tidak bisa undo/redo
- Sulit track perubahan state
- Rawan race condition di multi-threading

### AFTER (Fungsional, Pure) - CREATE

In [ ]:
def tambah_perangkat(perangkat_baru: dict, daftar_perangkat: list) -> list:
    return [*daftar_perangkat, perangkat_baru]

def demo_create_functional():
    devices = [
        {'id': 'SN-001', 'tipe': 'Sensor Suhu', 'status': 'Aktif'},
        {'id': 'AC-001', 'tipe': 'Aktuator', 'status': 'Aktif'}
    ]
    
    print("Sebelum tambah:", devices)
    
    new_device = {'id': 'SN-002', 'tipe': 'Sensor Kelembapan', 'status': 'Aktif'}
    updated_devices = tambah_perangkat(new_device, devices)
    
    print("Setelah tambah:", updated_devices)
    print("Original tidak berubah:", devices)

demo_create_functional()

Sebelum tambah: [{'id': 'SN-001', 'tipe': 'Sensor Suhu', 'status': 'Aktif'}, {'id': 'AC-001', 'tipe': 'Aktuator', 'status': 'Aktif'}]
Setelah tambah: [{'id': 'SN-001', 'tipe': 'Sensor Suhu', 'status': 'Aktif'}, {'id': 'AC-001', 'tipe': 'Aktuator', 'status': 'Aktif'}, {'id': 'SN-002', 'tipe': 'Sensor Kelembapan', 'status': 'Aktif'}]
Original tidak berubah: [{'id': 'SN-001', 'tipe': 'Sensor Suhu', 'status': 'Aktif'}, {'id': 'AC-001', 'tipe': 'Aktuator', 'status': 'Aktif'}]


### AFTER (Fungsional, Pure) - READ

In [ ]:
def filter_perangkat_by_status(status: str, daftar: list) -> list:
    return list(filter(lambda p: p['status'] == status, daftar))

def format_daftar_perangkat(daftar: list) -> list[str]:
    return [
        f"{i+1}. {p['id']:<12} | {p['tipe']:<20} | {p['status']}"
        for i, p in enumerate(daftar)
    ]

def demo_read_functional():
    devices = [
        {'id': 'SN-001', 'tipe': 'Sensor Suhu', 'status': 'Aktif'},
        {'id': 'AC-001', 'tipe': 'Aktuator', 'status': 'Maintenance'},
        {'id': 'SN-002', 'tipe': 'Sensor Kelembapan', 'status': 'Aktif'}
    ]
    
    aktif = filter_perangkat_by_status('Aktif', devices)
    print("Perangkat Aktif:")
    for line in format_daftar_perangkat(aktif):
        print(line)
    
    print("\nSemua Perangkat:")
    for line in format_daftar_perangkat(devices):
        print(line)

demo_read_functional()

Perangkat Aktif:
1. SN-001       | Sensor Suhu          | Aktif
2. SN-002       | Sensor Kelembapan    | Aktif

Semua Perangkat:
1. SN-001       | Sensor Suhu          | Aktif
2. AC-001       | Aktuator             | Maintenance
3. SN-002       | Sensor Kelembapan    | Aktif


### AFTER (Fungsional, Pure) - UPDATE

In [ ]:
def update_status_perangkat(device_id: str, status_baru: str, daftar: list) -> list:
    return [
        {**p, 'status': status_baru} if p['id'] == device_id else p
        for p in daftar
    ]

def update_perangkat_by_index(index: int, updates: dict, daftar: list) -> list:
    """Update perangkat di index tertentu dengan dict updates."""
    if 0 <= index < len(daftar):
        return [
            {**p, **updates} if i == index else p
            for i, p in enumerate(daftar)
        ]
    return daftar

def demo_update_functional():
    devices = [
        {'id': 'SN-001', 'tipe': 'Sensor Suhu', 'status': 'Aktif'},
        {'id': 'AC-001', 'tipe': 'Aktuator', 'status': 'Aktif'}
    ]
    
    print("Sebelum:", devices)
    
    updated = update_status_perangkat('SN-001', 'Maintenance', devices)
    print("Setelah update ID:", updated)
    
    updated2 = update_perangkat_by_index(1, {'status': 'Rusak', 'tipe': 'Aktuator V2'}, devices)
    print("Setelah update index:", updated2)
    
    print("Original tetap:", devices)

demo_update_functional()

Sebelum: [{'id': 'SN-001', 'tipe': 'Sensor Suhu', 'status': 'Aktif'}, {'id': 'AC-001', 'tipe': 'Aktuator', 'status': 'Aktif'}]
Setelah update ID: [{'id': 'SN-001', 'tipe': 'Sensor Suhu', 'status': 'Maintenance'}, {'id': 'AC-001', 'tipe': 'Aktuator', 'status': 'Aktif'}]
Setelah update index: [{'id': 'SN-001', 'tipe': 'Sensor Suhu', 'status': 'Aktif'}, {'id': 'AC-001', 'tipe': 'Aktuator V2', 'status': 'Rusak'}]
Original tetap: [{'id': 'SN-001', 'tipe': 'Sensor Suhu', 'status': 'Aktif'}, {'id': 'AC-001', 'tipe': 'Aktuator', 'status': 'Aktif'}]


### AFTER (Fungsional, Pure) - DELETE

In [ ]:
def hapus_perangkat_by_id(device_id: str, daftar: list) -> list:
    return [p for p in daftar if p['id'] != device_id]

def hapus_perangkat_by_index(index: int, daftar: list) -> list:
    """Hapus perangkat di index tertentu (immutable)."""
    if 0 <= index < len(daftar):
        return [p for i, p in enumerate(daftar) if i != index]
    return daftar

def hapus_perangkat_by_kondisi(predicate, daftar: list) -> list:
    """
    Hapus semua perangkat yang memenuhi kondisi.
    predicate: fungsi yang menerima dict perangkat, return bool
    """
    return [p for p in daftar if not predicate(p)]

def demo_delete_functional():
    devices = [
        {'id': 'SN-001', 'tipe': 'Sensor Suhu', 'status': 'Aktif'},
        {'id': 'AC-001', 'tipe': 'Aktuator', 'status': 'Rusak'},
        {'id': 'SN-002', 'tipe': 'Sensor Kelembapan', 'status': 'Aktif'}
    ]
    
    print("Original:", devices)
    
    after_delete_id = hapus_perangkat_by_id('AC-001', devices)
    print("\nSetelah hapus AC-001:", after_delete_id)
    
    after_delete_idx = hapus_perangkat_by_index(0, devices)
    print("Setelah hapus index 0:", after_delete_idx)
    
    after_delete_rusak = hapus_perangkat_by_kondisi(
        lambda p: p['status'] == 'Rusak',
        devices
    )
    print("Setelah hapus yang rusak:", after_delete_rusak)
    
    print("\nOriginal tetap:", devices)

demo_delete_functional()

Original: [{'id': 'SN-001', 'tipe': 'Sensor Suhu', 'status': 'Aktif'}, {'id': 'AC-001', 'tipe': 'Aktuator', 'status': 'Rusak'}, {'id': 'SN-002', 'tipe': 'Sensor Kelembapan', 'status': 'Aktif'}]

Setelah hapus AC-001: [{'id': 'SN-001', 'tipe': 'Sensor Suhu', 'status': 'Aktif'}, {'id': 'SN-002', 'tipe': 'Sensor Kelembapan', 'status': 'Aktif'}]
Setelah hapus index 0: [{'id': 'AC-001', 'tipe': 'Aktuator', 'status': 'Rusak'}, {'id': 'SN-002', 'tipe': 'Sensor Kelembapan', 'status': 'Aktif'}]
Setelah hapus yang rusak: [{'id': 'SN-001', 'tipe': 'Sensor Suhu', 'status': 'Aktif'}, {'id': 'SN-002', 'tipe': 'Sensor Kelembapan', 'status': 'Aktif'}]

Original tetap: [{'id': 'SN-001', 'tipe': 'Sensor Suhu', 'status': 'Aktif'}, {'id': 'AC-001', 'tipe': 'Aktuator', 'status': 'Rusak'}, {'id': 'SN-002', 'tipe': 'Sensor Kelembapan', 'status': 'Aktif'}]


**Penjelasan Transformasi CRUD:**

1. **CREATE**: Menggunakan spread operator `[*list, new_item]` untuk membuat list baru
2. **READ**: Menggunakan `filter()` dan list comprehension untuk transformasi deklaratif
3. **UPDATE**: Menggunakan comprehension dengan conditional `{**dict, key: new_val}` untuk merge immutable
4. **DELETE**: Menggunakan filter/comprehension untuk membuat list tanpa elemen tertentu

**Keuntungan Pure Functions:**
- **Testable**: Mudah unit test tanpa setup state global
- **Predictable**: Input sama → output sama (deterministik)
- **Composable**: Fungsi bisa digabungkan tanpa konflik
- **Thread-safe**: Tidak ada race condition karena immutable
- **Time-travel debugging**: Bisa undo/redo dengan mudah

---

## 4. Transformasi: Fungsi Utilitas (Validasi Input)

### BEFORE (Imperatif, Impure)
```python
def validasi_input_angka(prompt, rentang_maks):
    while True:
        pilihan = input(prompt)
        if pilihan.isdigit():
            pilihan_int = int(pilihan)
            if 1 <= pilihan_int <= rentang_maks:
                return pilihan_int
            else:
                print(f"Error: harus 1-{rentang_maks}")
        else:
            print("Error: harus angka")
```

**Masalah:**
- I/O dan validasi tercampur
- Side effect (print) di dalam loop
- Tidak bisa di-test tanpa mock

### AFTER (Fungsional, Pure)

In [ ]:
def validasi_angka_rentang(input_str: str, min_val: int, max_val: int) -> tuple[bool, int | None, str]:
    if not input_str.isdigit():
        return (False, None, "Input harus berupa angka")
    
    val = int(input_str)
    if not (min_val <= val <= max_val):
        return (False, None, f"Angka harus antara {min_val} dan {max_val}")
    
    return (True, val, "")

def validasi_dengan_kondisi(input_str: str, *validators) -> tuple[bool, str]:
    for validator in validators:
        is_valid, msg = validator(input_str)
        if not is_valid:
            return (False, msg)
    return (True, "")

def is_not_empty(s: str) -> tuple[bool, str]:
    return (len(s) > 0, "Input tidak boleh kosong") if not s else (True, "")

def is_digit(s: str) -> tuple[bool, str]:
    return (s.isdigit(), "Harus berupa angka") if not s.isdigit() else (True, "")

def demo_validasi_functional():
    test_inputs = ["5", "abc", "15", "0", ""]
    
    for inp in test_inputs:
        valid, value, msg = validasi_angka_rentang(inp, 1, 10)
        if valid:
            print(f"✓ '{inp}' valid → {value}")
        else:
            print(f"✗ '{inp}' invalid: {msg}")

demo_validasi_functional()

✓ '5' valid → 5
✗ 'abc' invalid: Input harus berupa angka
✗ '15' invalid: Angka harus antara 1 dan 10
✗ '0' invalid: Angka harus antara 1 dan 10
✗ '' invalid: Input harus berupa angka


---

## 5. Ringkasan Perbandingan: Imperatif vs Fungsional

### Tabel Perbandingan Karakteristik

| Aspek | BEFORE (Imperatif) | AFTER (Fungsional) |
|-------|-------------------|-------------------|
| **State Management** | Mutasi variabel global | Immutable, return new state |
| **Side Effects** | Print, input dalam fungsi | Terpisah di UI layer |
| **Testability** | Sulit (perlu mock I/O) | Mudah (pure function) |
| **Predictability** | Bergantung state global | Deterministik (input → output) |
| **Composability** | Sulit digabungkan | Mudah di-compose |
| **Debugging** | Sulit track state mutation | Jelas (setiap fungsi return baru) |
| **Concurrency** | Rawan race condition | Thread-safe (immutable) |
| **Code Reuse** | Terikat konteks spesifik | Generic, reusable |

### Teknik Transformasi yang Digunakan

1. **Spread Operator**: `{**dict, key: val}`, `[*list, item]` untuk copy + modify
2. **List Comprehension**: Deklaratif untuk filter/map/transform
3. **Higher-Order Functions**: `map()`, `filter()`, lambda untuk abstraksi
4. **Tuple Returns**: Mengembalikan multiple values (status, data, error)
5. **Separation of Concerns**: Logika bisnis terpisah dari I/O
6. **Pattern Matching**: Validasi menggunakan tuple unpacking

## 6. Demonstrasi End-to-End: Sistem Fungsional Lengkap

Berikut implementasi sistem lengkap dengan paradigma fungsional murni:

In [ ]:
# ===== SISTEM MANAJEMEN PROYEK IOT - VERSI FUNGSIONAL =====

# State Management dengan immutable data structures
from typing import NamedTuple, Callable
from functools import reduce

class AppState(NamedTuple):
    akun: dict
    profil: dict
    proyek: dict
    user_login: str | None = None

def register_user(state: AppState, user_id: str, pwd: str, nama: str, jabatan: str) -> AppState:
    """Register user baru, return state baru."""
    return AppState(
        akun={**state.akun, user_id: pwd},
        profil={**state.profil, user_id: {'Nama': nama, 'Jabatan': jabatan}},
        proyek={**state.proyek, user_id: []},
        user_login=state.user_login
    )

def login_user(state: AppState, user_id: str, pwd: str) -> AppState:
    """Login user jika valid, return state dengan user_login updated."""
    if autentikasi(user_id, pwd, state.akun):
        return state._replace(user_login=user_id)
    return state

def logout_user(state: AppState) -> AppState:
    """Logout user, return state dengan user_login = None."""
    return state._replace(user_login=None)

def add_device(state: AppState, device: dict) -> AppState:
    """Tambah perangkat untuk user yang login."""
    if state.user_login is None:
        return state
    
    user_devices = state.proyek.get(state.user_login, [])
    new_devices = [*user_devices, device]
    
    return state._replace(
        proyek={**state.proyek, state.user_login: new_devices}
    )

def update_device_status(state: AppState, device_id: str, new_status: str) -> AppState:
    if state.user_login is None:
        return state
    
    user_devices = state.proyek.get(state.user_login, [])
    updated_devices = update_status_perangkat(device_id, new_status, user_devices)
    
    return state._replace(
        proyek={**state.proyek, state.user_login: updated_devices}
    )

def delete_device(state: AppState, device_id: str) -> AppState:
    if state.user_login is None:
        return state
    
    user_devices = state.proyek.get(state.user_login, [])
    updated_devices = hapus_perangkat_by_id(device_id, user_devices)
    
    return state._replace(
        proyek={**state.proyek, state.user_login: updated_devices}
    )

# Demo sistem fungsional end-to-end
def demo_sistem_fungsional():
    # Initial state
    state = AppState(
        akun={'admin': 'admin123'},
        profil={'admin': {'Nama': 'Admin', 'Jabatan': 'PM'}},
        proyek={'admin': []}
    )
    
    print("=== Demo Sistem Fungsional ===\n")
    
    # 1. Register user baru
    state = register_user(state, 'user01', 'pass01', 'Budi', 'Engineer')
    print("1. Setelah register user01:")
    print(f"   Akun: {list(state.akun.keys())}")
    
    # 2. Login
    state = login_user(state, 'user01', 'pass01')
    print(f"\n2. Login user01:")
    print(f"   User login: {state.user_login}")
    
    # 3. Tambah perangkat
    state = add_device(state, {'id': 'SN-001', 'tipe': 'Sensor Suhu', 'status': 'Aktif'})
    state = add_device(state, {'id': 'AC-001', 'tipe': 'Aktuator', 'status': 'Aktif'})
    print(f"\n3. Setelah tambah 2 perangkat:")
    print(f"   Devices: {state.proyek['user01']}")
    
    # 4. Update status
    state = update_device_status(state, 'SN-001', 'Maintenance')
    print(f"\n4. Setelah update status SN-001:")
    print(f"   Devices: {state.proyek['user01']}")
    
    # 5. Hapus perangkat
    state = delete_device(state, 'AC-001')
    print(f"\n5. Setelah hapus AC-001:")
    print(f"   Devices: {state.proyek['user01']}")
    
    # 6. Logout
    state = logout_user(state)
    print(f"\n6. Setelah logout:")
    print(f"   User login: {state.user_login}")
    
    print("\nSemua operasi dilakukan secara immutable dan pure")

demo_sistem_fungsional()

=== Demo Sistem Fungsional ===

1. Setelah register user01:
   Akun: ['admin', 'user01']

2. Login user01:
   User login: user01

3. Setelah tambah 2 perangkat:
   Devices: [{'id': 'SN-001', 'tipe': 'Sensor Suhu', 'status': 'Aktif'}, {'id': 'AC-001', 'tipe': 'Aktuator', 'status': 'Aktif'}]

4. Setelah update status SN-001:
   Devices: [{'id': 'SN-001', 'tipe': 'Sensor Suhu', 'status': 'Maintenance'}, {'id': 'AC-001', 'tipe': 'Aktuator', 'status': 'Aktif'}]

5. Setelah hapus AC-001:
   Devices: [{'id': 'SN-001', 'tipe': 'Sensor Suhu', 'status': 'Maintenance'}]

6. Setelah logout:
   User login: None

✓ Semua operasi dilakukan secara immutable dan pure!


---

## 7. Kesimpulan dan Pembelajaran

### Prinsip Utama Paradigma Fungsional yang Diterapkan

1. **Pure Functions**
   - Input sama → Output sama (deterministic)
   - Tidak ada side effect (no mutation, no I/O)
   - Mudah di-test dan di-reason

2. **Immutability**
   - Data tidak pernah diubah in-place
   - Selalu membuat struktur baru dengan `{...}`, `[...]`
   - Mencegah bug dari shared mutable state

3. **Declarative Code**
   - Menggunakan comprehension, `map`, `filter`, `reduce`
   - Fokus pada "apa" bukan "bagaimana"
   - Lebih ekspresif dan ringkas

4. **Separation of Concerns**
   - Logika bisnis terpisah dari I/O
   - Fungsi kecil, single responsibility
   - Composable dan reusable

5. **Higher-Order Functions**
   - Fungsi sebagai first-class citizen
   - Menerima/mengembalikan fungsi
   - Abstraksi yang powerful

### Keuntungan yang Didapat

✓ **Testability**: Unit test tanpa mock, mudah verify  
✓ **Maintainability**: Perubahan lokal, tidak affect global  
✓ **Predictability**: Tidak ada "spooky action at distance"  
✓ **Concurrency**: Thread-safe by design (immutable)  
✓ **Debugging**: State transitions jelas, bisa time-travel  
✓ **Refactoring**: Aman karena no side effects

### Trade-offs yang Perlu Dipertimbangkan

⚠ **Performance**: Copy data bisa lebih lambat untuk struktur besar  
⚠ **Memory**: Bisa lebih boros (multiple copies)  
⚠ **Learning Curve**: Butuh mindset shift dari imperative  
⚠ **Verbosity**: Kadang lebih verbose untuk task sederhana

### Kapan Menggunakan Pendekatan Fungsional?

**Ideal untuk:**
- Business logic yang kompleks
- Aplikasi yang butuh reliable state management
- Sistem distributed/concurrent
- Kode yang perlu sering di-test

**Kurang ideal untuk:**
- Operasi I/O intensive (database, file)
- Performance-critical dengan data besar
- Prototype cepat yang simple

---

**Fatih - 132**  
Modul 2: Transformasi Paradigma Fungsional  
Program Manajemen Proyek IoT